![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [ ]:
model_id = "sdaia/allam-1-13b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "decoding_method": "sample",
    "max_new_tokens": 200,
    "stop_sequences": ["<end>"],
    "temperature": 0.72,
    "top_k": 45,
    "top_p": 0.98,
    "repetition_penalty": 1.03
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [ ]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)


## Defining the inferencing input
Foundation model inferencing API accepts a natural language input that it will use
to provide the natural language response. The API is sensitive to formatting. Input
structure, presence of training steps (one-shot, two-shot learning etc.), as well
as phrasing all influence the final response and belongs to the emerging discipline of
Prompt Engineering.

Let us provide the input we got from the Prompt Lab:


In [ ]:
diff_level =  "" 
lvl_prep =  "" 
correct_example =  "" 
prep_letter_example_and_rule =  "" 

prompt_input = f"""أحرف الجر: من، إلى، عن، على، ك، ل، ب.
اكتب جملة منطقية، صحيحة المعنى وصرف الكلمات، مشكولة باللغة العربية تناسب متعلم من المستوى {diff_level}.
الجملة يجب أن تحتوي على حرف الجر الآتي: {lvl_prep} .
يمكن أن يأتي حرف الجر في بداية الجملة أو وسطها أو نهايتها.
أجعل بعض الجمل تبدأ باسم وبعض الجمل الأخرى تبدأ بفعل. 
اكتب حرف الجر المذكور في مكان الإجابة الصحيحة.
{prep_letter_example_and_rule}
شكل الخرج:
اطبع الجملة sentence والإجابة الصحيحة correct_answer على شكل كيان JSON:
<start>
[{{ "sentence": "...", "correct_answer": "..." }}]
<end>


Input: اكتب جملة منطقية، صحيحة المعنى وصرف الكلمات، مشكولة، باللغة العربية تناسب متعلم من المستوى المبتدئ.
الجملة يجب أن تحتوي على حرف الجر الآتي: على.
يمكن أن يأتي حرف الجر في بداية الجملة أو وسطها أو نهايتها.
أجعل بعض الجمل تبدأ باسم وبعض الجمل الأخرى تبدأ بفعل. 
اكتب حرف الجر المذكور في مكان الإجابة الصحيحة.
Output: <start>
[{{
"sentence": "جلس الطفل على الكرسي",
"correct_answer": "على"
}}]
<end>

Input: اكتب جملة منطقية، صحيحة المعنى وصرف الكلمات، مشكولة، باللغة العربية تناسب متعلم من المستوى المتقدم.
الجملة يجب أن تحتوي على حرف الجر الآتي: ل.
يمكن أن يأتي حرف الجر في بداية الجملة أو وسطها أو نهايتها.
أجعل بعض الجمل تبدأ باسم وبعض الجمل الأخرى تبدأ بفعل. 
اكتب حرف الجر المذكور في مكان الإجابة الصحيحة.
إذا كان حرف الجر هو : ل، فالإجابة الصحيحة هي حرف الجر بالإضافة إلى الاسم المجرور الذي يليه. مثال: للطلاب.
Output: <start>
[{{
"sentence": "أحببت أن أجلس على الشاطئ لأستمتع بمنظر البحر الجميل",
"correct_answer": "لأستمتع"
}}]
<end>

Input: اكتب جملة منطقية، صحيحة المعنى وصرف الكلمات، مشكولة، باللغة العربية تناسب متعلم من المستوى المتقدم.
الجملة يجب أن تحتوي على حرف الجر الآتي: ب.
يمكن أن يأتي حرف الجر في بداية الجملة أو وسطها أو نهايتها.
أجعل بعض الجمل تبدأ باسم وبعض الجمل الأخرى تبدأ بفعل. 
اكتب حرف الجر المذكور في مكان الإجابة الصحيحة.
إذا كان حرف الجر هو : ب، فالإجابة الصحيحة هي حرف الجر بالإضافة إلى الاسم المجرور الذي يليه. مثال: بالدماء.
Output: <start>
[{{
"sentence": "شعرت بالراحة عندما جلست تحت ظل الشجرة",
"correct_answer": "بالراحة"
}}]
<end>

Input: اكتب جملة منطقية، صحيحة المعنى وصرف الكلمات، مشكولة، باللغة العربية تناسب متعلم من المستوى المبتدئ.
الجملة يجب أن تحتوي على حرف الجر الآتي: في.
يمكن أن يأتي حرف الجر في بداية الجملة أو وسطها أو نهايتها.
أجعل بعض الجمل تبدأ باسم وبعض الجمل الأخرى تبدأ بفعل. 
اكتب حرف الجر المذكور في مكان الإجابة الصحيحة.
Output: <start>
[{{
"sentence": "ذهبنا إلى المدرسة في الصباح الباكر",
"correct_answer": "في"
}}]
<end>

Input: اكتب جملة منطقية، صحيحة المعنى وصرف الكلمات، مشكولة، باللغة العربية تناسب متعلم من المستوى المتقدم.
الجملة يجب أن تحتوي على حرف الجر الآتي: ك.
يمكن أن يأتي حرف الجر في بداية الجملة أو وسطها أو نهايتها.
أجعل بعض الجمل تبدأ باسم وبعض الجمل الأخرى تبدأ بفعل. 
اكتب حرف الجر المذكور في مكان الإجابة الصحيحة.
إذا كان حرف الجر هو : ك، فالإجابة الصحيحة هي حرف الجر بالإضافة إلى الاسم المجرور الذي يليه. مثال: كالفهد.
Output: <start>
[{{
"sentence": "تحركت السيارة بسرعة كبيرة كسيارة رياضية",
"correct_answer": "كسيارة"
}}]
<end>

Input: اكتب جملة منطقية، صحيحة المعنى وصرف الكلمات، مشكولة، باللغة العربية تناسب متعلم من المستوى المبتدئ.
الجملة يجب أن تحتوي على حرف الجر الآتي: إلى.
يمكن أن يأتي حرف الجر في بداية الجملة أو وسطها أو نهايتها.
أجعل بعض الجمل تبدأ باسم وبعض الجمل الأخرى تبدأ بفعل. 
اكتب حرف الجر المذكور في مكان الإجابة الصحيحة.
Output: <start>
[{{
"sentence": "وصل الطالب إلى الفصل متأخرا",
"correct_answer": "إلى"
}}]
<end>

Input: اكتب جملة منطقية، صحيحة المعنى وصرف الكلمات، مشكولة، باللغة العربية تناسب متعلم من المستوى المبتدئ.
الجملة يجب أن تحتوي على حرف الجر الآتي: من.
يمكن أن يأتي حرف الجر في بداية الجملة أو وسطها أو نهايتها.
أجعل بعض الجمل تبدأ باسم وبعض الجمل الأخرى تبدأ بفعل. 
اكتب حرف الجر المذكور في مكان الإجابة الصحيحة.
Output: <start>
[{{
"sentence": "خرجت من البيت بعد أن تناولت الإفطار",
"correct_answer": "من"
}}]
<end>

Input: اكتب جملة منطقية، صحيحة المعنى وصرف الكلمات، مشكولة، باللغة العربية تناسب متعلم من المستوى المتقدم.
الجملة يجب أن تحتوي على حرف الجر الآتي: عن.
يمكن أن يأتي حرف الجر في بداية الجملة أو وسطها أو نهايتها.
أجعل بعض الجمل تبدأ باسم وبعض الجمل الأخرى تبدأ بفعل. 
اكتب حرف الجر المذكور في مكان الإجابة الصحيحة.
Output: <start>
[{{
"sentence": "تحدث المعلم عن أهمية القراءة وتنمية مهارات التفكير الناقد",
"correct_answer": "عن"
}}]
<end>

Input: اكتب جملة منطقية، صحيحة المعنى وصرف الكلمات، مشكولة، باللغة العربية تناسب متعلم من المستوى {diff_level}.
الجملة يجب أن تحتوي على حرف الجر الآتي: {lvl_prep}.
يمكن أن يأتي حرف الجر في بداية الجملة أو وسطها أو نهايتها.
أجعل بعض الجمل تبدأ باسم وبعض الجمل الأخرى تبدأ بفعل. 
اكتب حرف الجر المذكور في مكان الإجابة الصحيحة.
{prep_letter_example_and_rule}


Output:"""


## Execution
Let us now use the defined Model object and pair it with input and
generate the response:


In [ ]:
print("Submitting generation request...")
generated_response = model.generate_text(prompt=prompt_input, guardrails=False)
print(generated_response)


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  